In [128]:
import xlwings as xw
import pandas as pd
import numpy as np


In [129]:
excel_file = r"autosarexcel.xlsx"
app = xw.App(visible=False, add_book=False)
workbook = app.books.open(excel_file)
main_sheet = workbook.sheets["AP9200_VCM_rev02"]
last_row = main_sheet.used_range.last_cell.row
last_col = main_sheet.used_range.last_cell.column
begin_range = main_sheet.range("A2")
end_range = main_sheet.range((last_row, last_col))
main_range = main_sheet.range(begin_range, end_range)
main_df = main_range.options(pd.DataFrame, index=False, header=True).value

In [130]:
main_df

,A,B,C,D,E,F,G,H,I,J,...,AL,AM,AN,AO,AP,AQ,AR,AS,AT,AU
0,None,TSC1RECUEMdap_A,None,None,None,None,TSC1_RECU_EMSdap,None,0xC001001,None,...,None,None,None,None,K_TSC1RECUEMdap_A,can_gmlan_TSC1RECUEMdap_A,None,can_rx_TSC1RECUEMdap_A,can_invalid_TSC1RECUEMdap_A,None
1,None,TSC1RECUEMdap_B,None,None,None,None,TSC1_RECU_EMSdap,None,0xC001001,None,...,None,None,None,None,K_TSC1RECUEMdap_B,can_gmlan_TSC1RECUEMdap_B,None,can_rx_TSC1RECUEMdap_B,can_invalid_TSC1RECUEMdap_B,None
2,None,TSC1RECUEMdap_C,None,None,None,None,TSC1_RECU_EMSdap,None,0xC001001,None,...,None,None,None,None,K_TSC1RECUEMdap_C,can_gmlan_TSC1RECUEMdap_C,None,can_rx_TSC1RECUEMdap_C,can_invalid_TSC1RECUEMdap_C,None
3,None,TSC1RECUEMdap_D,None,None,None,None,TSC1_RECU_EMSdap,None,0xC001001,None,...,None,None,None,None,K_TSC1RECUEMdap_D,can_gmlan_TSC1RECUEMdap_D,None,can_rx_TSC1RECUEMdap_D,can_invalid_TSC1RECUEMdap_D,None
4,None,TSC1RECUEMdap_E,None,None,None,None,TSC1_RECU_EMSdap,None,0xC001001,None,...,None,None,None,None,K_TSC1RECUEMdap_E,can_gmlan_TSC1RECUEMdap_E,None,can_rx_TSC1RECUEMdap_E,can_invalid_TSC1RECUEMdap_E,None
5,None,VCMP19C02_A,None,None,None,None,VCM_P19_C02,None,0x424,None,...,None,None,None,None,K_VCMP19C02_A,can_gmlan_VCMP19C02_A,None,can_rx_VCMP19C02_A,can_invalid_VCMP19C02_A,None
6,None,VCMP19C02_B,None,None,None,None,VCM_P19_C02,None,0x424,None,...,None,None,None,None,K_VCMP19C02_B,can_gmlan_VCMP19C02_B,None,can_rx_VCMP19C02_B,can_invalid_VCMP19C02_B,None
7,None,VCMP19C02_C,None,None,None,None,VCM_P19_C02,None,0x424,None,...,None,None,None,None,K_VCMP19C02_C,can_gmlan_VCMP19C02_C,None,can_rx_VCMP19C02_C,can_invalid_VCMP19C02_C,None
8,None,VCMP19C02_D,None,None,None,None,VCM_P19_C02,None,0x424,None,...,None,None,None,None,K_VCMP19C02_D,can_gmlan_VCMP19C02_D,None,can_rx_VCMP19C02_D,can_invalid_VCMP19C02_D,None


In [131]:
tx_df = main_df[main_df["AA"] == "TX"]
tx_list_tmp = list(tx_df["I"].values)
tx_list = list(set(tx_list_tmp))
tx_list.sort(key=tx_list_tmp.index)

rx_df = main_df[main_df["AA"] == "RX"]
rx_list_tmp = list(rx_df["I"].values)
rx_list = list(set(rx_list_tmp))
rx_list.sort(key=rx_list_tmp.index)

with xw.App(visible=True, add_book=False) as app:

    def set_xls_format(sht):
        sht.range("A1").expand("right").color = (204, 255, 255)
        sht.range("A1").expand("right").font.bold = True
        sht.range("A1").expand("right").font.size = 13
        sht.used_range.api.Borders(8).LineStyle = 1
        sht.used_range.api.Borders(9).LineStyle = 1
        sht.used_range.api.Borders(7).LineStyle = 1
        sht.used_range.api.Borders(10).LineStyle = 1
        sht.used_range.api.Borders(12).LineStyle = 1
        sht.used_range.api.Borders(11).LineStyle = 1
        sht.used_range.api.HorizontalAlignment = -4152
        sht.used_range.api.VerticalAlignment = -4107
        sht.autofit()

    workbook = app.books.add()

    #################
    # TX
    #################
    # TX Summary
    tx_idx_sht = workbook.sheets.add(after=workbook.sheets.count)
    tx_idx_sht.name = "TxNormal"
    tx_summary_header = [["No.", "FrameID", "J1939/ISO"]]
    tx_summary_info = []
    # TX Msg
    for tx_idx, i_tx in enumerate(tx_list):
        tx_msg_df = tx_df[tx_df["I"] == i_tx]
        tx_msg_attr = list(set(tx_msg_df["AH"]))
        tx_msg_type = ""
        if len(tx_msg_attr) == 1:
            if tx_msg_attr[0]:
                tx_msg_type = "J1939"
            else:
                tx_msg_type = "ISO"
        tx_summary_info.append([tx_idx + 1, i_tx, tx_msg_type])
        tx_msg_info = []
        for _, i_sig in tx_msg_df.iterrows():
            i_sig_info = [
                "",
                i_sig["AJ"],
                "",
                i_sig["R"],
                i_sig["S"],
                i_sig["T"],
                i_sig["U"],
                "",
            ]
            tx_msg_info.append(i_sig_info)
        tx_msg_sht = workbook.sheets.add(after=workbook.sheets.count)
        tx_msg_sht.name = i_tx
        tx_msg_header = [
            [
                "Interface",
                "Signal",
                "Invalid Status",
                "Factor",
                "Offset",
                "Min",
                "Max",
                "SignalId",
            ]
        ]
        tx_msg_sht.range("A1").value = tx_msg_header + tx_msg_info
        set_xls_format(tx_msg_sht)

    tx_idx_sht.range("A1").value = tx_summary_header + tx_summary_info
    set_xls_format(tx_idx_sht)

    #################
    # RX
    #################
    rx_idx_sht = workbook.sheets.add(after=workbook.sheets.count)
    rx_idx_sht.name = "RxNormal&RxTimeout"
    rx_summary_header = [
        [
            "No.",
            "FrameID",
            "J1939/ISO",
            "Timeout flag",
            "SIGNALGROUP NAME",
            "IFLayer_rx_cmplt_xxx",
            "IFLayer_rx_timeout_xxx",
            "can_gmlan_cmplt_xxx",
            "can_gmlan_rx_timeout_xxx",
        ]
    ]
    rx_summary_info = []
    for rx_idx, i_rx in enumerate(rx_list):
        rx_msg_df = rx_df[rx_df["I"] == i_rx]
        rx_msg_attr = list(set(rx_msg_df["AH"]))
        rx_msg_type = ""
        if len(rx_msg_attr) == 1:
            if rx_msg_attr[0]:
                rx_msg_type = "J1939"
            else:
                rx_msg_type = "ISO"
        rx_summary_info.append(
            [
                rx_idx + 1,
                i_rx,
                rx_msg_type,
                "",
                "",
                "IFLayer_rx_cmplt_" + i_rx,
                "IFLayer_rx_timeout_" + i_rx,
                "can_gmlan_cmplt_" + i_rx,
                "can_gmlan_rx_timeout_" + i_rx,
            ]
        )
        rx_msg_info = []
        for _, i_sig in tx_msg_df.iterrows():
            i_sig_info = [
                "SYCANC_COM2AP_" + i_rx + "_rx_" + i_sig["AJ"],
                "IFLayer_" + i_rx + "_rx_" + i_sig["AJ"],
                i_sig["R"],
                i_sig["S"],
                i_sig["T"],
                i_sig["U"],
                "",
                i_sig["AP"],
                i_sig["AQ"],
                "",
                "",
                "",
                "",
                "",
                "",
                "",
                "",
                "",
                ""
            ]
            rx_msg_info.append(i_sig_info)
        rx_msg_sht = workbook.sheets.add(after=workbook.sheets.count)
        rx_msg_sht.name = i_rx
        rx_msg_header = [
            [
                "Signal Index",
                "IFLayer_Signal",
                "Factor",
                "Offset",
                "Min",
                "Max",
                "Invalid Status",
                "Error Indicator Value",
                "ISO CAN Receive Parameter Invalid Status",
                "ISO CAN Receive Parameter",
                "2-5-4 Requeset Paramter Select Flag",
                "J1939 CAN Receive Parameter Invalid Status",
                "J1939 CAN Receive Parameter",
                "RxSel Parameter Invalid Status",
                "RxSel Parameter",
                "3-1-1 Requeset Paramter Select Flag",
                "can_rqst_xxx",
                "can_invalid_can_rqst_xxx",
                "interface",
            ]
        ]
        rx_msg_sht.range("A1").value = rx_msg_header + rx_msg_info
        set_xls_format(rx_msg_sht)

    rx_idx_sht.range("A1").value = rx_summary_header + rx_summary_info
    set_xls_format(rx_idx_sht)

    workbook.sheets["Sheet1"].delete()
    workbook.save(r"Output.xlsx")